In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


# Built-in Tool - DuckDuckGo Search

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("top news in Nepal today")

print(results)

Kantipur Daily is Nepal's no 1 Nepali news portal and Kantipur Daily's official website. This delivers the latest breaking news and information on latest top stories, national, international, politics, sports, business, finance, entertainment, photo-gallery, audio, video and more. Nepal's No.1 English Daily Newspaper | Nepal News, Latest Politics, Business, World, Sports, Entertainment, Travel, Life Style News - Kathmandu "The Himalayan Post" is Nepal's No. 1 Nepali Online News Portal and official website. It provides the latest breaking news and updates. ... Check out today's news coverage live with photos on thehimalayanpost.com. If you are looking for the latest breaking news and information on the top stories, business, weather, politics, entertainment ... At least seven teachers and a few policemen were injured in Kathmandu city of Nepal on Sunday (April 27, 2025) as clashes erupted between the two sides during a demonstration for education reforms ... Nepali riot police lobbed te

In [3]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


# Built-in Tool - Shell Tool

In [4]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke("ls")

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# Custom Tools

In [5]:
from langchain_core.tools import tool

In [6]:
# step 1 create a function
def multiply(a, b):
  """Multiply two numbers"""
  return a*b

In [18]:
# step 2 add type hints
def multiply(a:int, b:int) -> int:
  """Multiply two numbers"""
  return a*b

In [13]:
# step 3 add tool decorator
@tool
def multiply(a:int, b:int) -> int:
  """Multiply two numbers"""
  return a*b

In [14]:
result = multiply.invoke({"a":3,"b":5})

In [15]:
print(result)

15


In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# Method 2 - Using StructuredTool

In [19]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [24]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True, description = "The first number to add")
  b : int = Field(required = True, description = "The second number to add")

In [25]:
def multiply_func(a:int, b:int) -> int:
  return a * b

In [26]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "Multiply",
    description = "Multiply two numbers",
    args_schema = MultiplyInput
)

In [27]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
Multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# Method 3 - Using BaseTool Class

In [31]:
from langchain.tools import BaseTool
from typing import Type

In [32]:
# arg schema using Pydantic

class MultiplyInput(BaseModel):
  a : int = Field(required = True, description = "The first number to add")
  b : int = Field(required = True, description = "The second number to add")

In [33]:
class MultiplyTool(BaseTool):
  name : str = "multiply"
  description : str = "Multiply two numbers"

  args_schema : Type[BaseModel] = MultiplyInput

  def _run(self, a:int, b:int) -> int:
    return a * b

In [34]:
multiply_tool = MultiplyTool()

In [35]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# Toolkit

In [36]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
  """Add two numbers"""
  return a + b

@tool
def multiply(a: int, b: int) -> int:
  """Multiply two numbers"""
  return a * b

In [37]:
class MathToolkit:
  def get_tools(self):
    return [add, multiply]

In [38]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
